# Counter-Example: Sensitivity of parameter `r`

In [ ]:
import os
import random
import time

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import torch
from torch.distributions import Bernoulli

from optimizers.generic_adam_cer import GenericAdamCER

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Testing with one value of r

Sources referenced:

1. https://github.com/wikaiqi/AMSGradpytorch/blob/master/AdamBeyond.ipynb
2. https://github.com/tamirbennatan/Adam-Convergence/blob/15eb471e8d2a480e91cd7c9777c8e6492cc20eea/Synthetic/Synthetic.ipynb
3. https://github.com/zshanggu/zeyu-Reproducing-Convergence-of-ADAM-and-Beyond/blob/master/synthetic_case_experiment.ipynb
4. https://fdlm.github.io/post/amsgrad/

In [ ]:
x = 0.5  # stochastic setting
r_value = 0.5
F = (-1, 1)
optimizer = GenericAdamCER(x_init=[x], F=F, r=r_value, device=device)

In [ ]:
num_iterations = int(1e7)
max_grad = 1010
min_grad = -10
probability = 0.01
log_every = 1e5
Bern_exp = Bernoulli(0.01)

regret = 0
regrets = []
x_values = []

In [ ]:
def ft_sto(bern, max_grad, min_grad):
    probr = bern.sample().item()  # Convert to scalar
    grad = (probr * max_grad) + ((1 - probr) * min_grad)
    return grad, probr

In [ ]:
def fmin_sto(probr, max_grad, min_grad):
    return -max_grad if probr > 0.5 else -min_grad

In [ ]:
for t in range(1, num_iterations + 1):
    grad, probr = ft_sto(Bern_exp, max_grad, min_grad)

    x = optimizer.step(grad)

    # Compute loss at the current x
    loss = grad * x

    # Compute the best achievable loss (min_loss) for the current gradient
    min_loss = fmin_sto(probr, max_grad, min_grad)

    regret += loss - min_loss
    regrets.append(regret / t)
    x_values.append(x)

    if t % log_every == 0:
        print(f"Iteration {t}, x: {x}, Average Regret: {regrets[-1]:.6f}")

Output from above execution:

```bash
Iteration 100000, x: 0.408403217792511, Average Regret: 0.679894
Iteration 200000, x: 0.73151034116745, Average Regret: 0.327554
Iteration 300000, x: 0.7130993604660034, Average Regret: 0.478726
Iteration 400000, x: 0.8762974143028259, Average Regret: 0.452666
Iteration 500000, x: 0.903300404548645, Average Regret: 0.232564
Iteration 600000, x: 0.9597113728523254, Average Regret: 0.160154
Iteration 700000, x: 0.7969937920570374, Average Regret: 0.254583
Iteration 800000, x: 0.8058254718780518, Average Regret: 0.330932
Iteration 900000, x: 0.9837645292282104, Average Regret: 0.319086
Iteration 1000000, x: 0.9776531457901001, Average Regret: 0.310313
Iteration 1100000, x: 0.7969663739204407, Average Regret: 0.383424
Iteration 1200000, x: 0.792559802532196, Average Regret: 0.373279
Iteration 1300000, x: 0.9893298745155334, Average Regret: 0.346244
Iteration 1400000, x: 0.9843884706497192, Average Regret: 0.235595
Iteration 1500000, x: 0.9709327220916748, Average Regret: 0.189795
Iteration 1600000, x: 0.9757027626037598, Average Regret: 0.221340
Iteration 1700000, x: 0.9723968505859375, Average Regret: 0.217178
...
Iteration 8600000, x: 0.9907640814781189, Average Regret: 0.195647
Iteration 8700000, x: 0.9727708697319031, Average Regret: 0.193760
Iteration 8800000, x: 0.9723454117774963, Average Regret: 0.195151
Iteration 8900000, x: 0.9868977069854736, Average Regret: 0.193529
Iteration 9000000, x: 0.9972649812698364, Average Regret: 0.182256
Iteration 9100000, x: 0.9518070220947266, Average Regret: 0.187897
Iteration 9200000, x: 0.962496817111969, Average Regret: 0.193948
Iteration 9300000, x: 0.9991404414176941, Average Regret: 0.177586
Iteration 9400000, x: 0.9327308535575867, Average Regret: 0.190621
Iteration 9500000, x: 0.9611507654190063, Average Regret: 0.195493
Iteration 9600000, x: 0.9465175271034241, Average Regret: 0.205331
Iteration 9700000, x: 0.982431948184967, Average Regret: 0.209857
Iteration 9800000, x: 0.9125189185142517, Average Regret: 0.227702
Iteration 9900000, x: 0.9906120896339417, Average Regret: 0.219302
Iteration 10000000, x: 0.9850069284439087, Average Regret: 0.220890
```

In [ ]:
data = pd.DataFrame({"Iteration": range(1, len(regrets) + 1), "Average Regret": regrets, "x Values": x_values})

In [ ]:
plt.figure(figsize=(15, 5))

plt.subplot(1, 2, 1)
sns.lineplot(data=data, x="Iteration", y="Average Regret", color="blue")
plt.title("Average Regret over Iterations", fontsize=14)
plt.xlabel("Iterations", fontsize=12)
plt.ylabel("Average Regret", fontsize=12)

plt.subplot(1, 2, 2)
sns.lineplot(data=data, x="Iteration", y="x Values", color="green")
plt.title("x Values over Iterations", fontsize=14)
plt.xlabel("Iterations", fontsize=12)
plt.ylabel("x", fontsize=12)

plt.tight_layout()
plt.show()

## Generic Adam - Sensitivity of `r`

In [ ]:
results = {}

In [ ]:
r_values = [0, 0.25, 0.5, 0.75, 1.0]

num_iterations = int(6e6)
log_interval = int(1e6)

In [ ]:
max_grad = 1010
min_grad = -10
probability = 0.01
F = (-1, 1)

In [ ]:
for r_value in r_values:
    print(f"Running experiment for r = {r_value}")

    x = 0.5
    optimizer = GenericAdamCER(x_init=[x], F=F, r=r_value, device=device)
    regret = 0
    regrets = []
    x_values = []
    iteration_times = []

    start_log_time = time.time()

    for t in range(1, num_iterations + 1):
        grad, probr = ft_sto(Bern_exp, max_grad, min_grad)
        x = optimizer.step(grad)

        # Compute loss at the current x
        loss = grad * x

        # Compute the best achievable loss (min_loss) for the current gradient
        min_loss = fmin_sto(probr, max_grad, min_grad)

        regret += loss - min_loss
        regrets.append(regret / t)
        x_values.append(x)

        if t % log_interval == 0:
            total_log_time = time.time() - start_log_time
            print(
                f"Iteration {t}, r = {r_value}, Average Regret: {regrets[-1]:.6f}, "
                f"Total Time for Last {log_interval} Iterations: {total_log_time:.4f}s"
            )
            start_log_time = time.time()

    results[r_value] = pd.DataFrame(
        {
            "Iteration": range(1, num_iterations + 1),
            "Average Regret": regrets,
            "x Values": x_values,
        }
    )

print("Experiments completed.")

Output:
```bash
Running experiment for r = 0
Iteration 1000000, r = 0, Average Regret: 0.346997, Total Time for Last 1000000 Iterations: 306.4111s
Iteration 2000000, r = 0, Average Regret: 0.342975, Total Time for Last 1000000 Iterations: 305.7375s
Iteration 3000000, r = 0, Average Regret: 0.349787, Total Time for Last 1000000 Iterations: 307.4160s
Iteration 4000000, r = 0, Average Regret: 0.395089, Total Time for Last 1000000 Iterations: 308.2684s
Iteration 5000000, r = 0, Average Regret: 0.387406, Total Time for Last 1000000 Iterations: 310.0230s
Iteration 6000000, r = 0, Average Regret: 0.419485, Total Time for Last 1000000 Iterations: 306.4983s
Running experiment for r = 0.25
Iteration 1000000, r = 0.25, Average Regret: 0.096384, Total Time for Last 1000000 Iterations: 305.3474s
Iteration 2000000, r = 0.25, Average Regret: 0.407986, Total Time for Last 1000000 Iterations: 304.7093s
Iteration 3000000, r = 0.25, Average Regret: 0.464050, Total Time for Last 1000000 Iterations: 307.2978s
Iteration 4000000, r = 0.25, Average Regret: 0.555298, Total Time for Last 1000000 Iterations: 307.0189s
Iteration 5000000, r = 0.25, Average Regret: 0.548092, Total Time for Last 1000000 Iterations: 307.6765s
Iteration 6000000, r = 0.25, Average Regret: 0.480102, Total Time for Last 1000000 Iterations: 308.0322s
Running experiment for r = 0.5
Iteration 1000000, r = 0.5, Average Regret: 0.665765, Total Time for Last 1000000 Iterations: 307.2238s
Iteration 2000000, r = 0.5, Average Regret: 0.638380, Total Time for Last 1000000 Iterations: 306.4598s
Iteration 3000000, r = 0.5, Average Regret: 0.590844, Total Time for Last 1000000 Iterations: 305.6680s
Iteration 4000000, r = 0.5, Average Regret: 0.569999, Total Time for Last 1000000 Iterations: 308.1390s
Iteration 5000000, r = 0.5, Average Regret: 0.552003, Total Time for Last 1000000 Iterations: 305.6050s
Iteration 6000000, r = 0.5, Average Regret: 0.523649, Total Time for Last 1000000 Iterations: 306.1684s
Running experiment for r = 0.75
Iteration 1000000, r = 0.75, Average Regret: 0.037433, Total Time for Last 1000000 Iterations: 308.4186s
Iteration 2000000, r = 0.75, Average Regret: 0.113313, Total Time for Last 1000000 Iterations: 308.6996s
Iteration 3000000, r = 0.75, Average Regret: 0.213503, Total Time for Last 1000000 Iterations: 306.7598s
Iteration 4000000, r = 0.75, Average Regret: 0.271067, Total Time for Last 1000000 Iterations: 310.3542s
Iteration 5000000, r = 0.75, Average Regret: 0.338192, Total Time for Last 1000000 Iterations: 306.8157s
Iteration 6000000, r = 0.75, Average Regret: 0.353814, Total Time for Last 1000000 Iterations: 308.2322s
Running experiment for r = 1.0
Iteration 1000000, r = 1.0, Average Regret: 0.494580, Total Time for Last 1000000 Iterations: 307.2563s
Iteration 2000000, r = 1.0, Average Regret: 0.537212, Total Time for Last 1000000 Iterations: 306.9033s
Iteration 3000000, r = 1.0, Average Regret: 0.444726, Total Time for Last 1000000 Iterations: 305.7439s
Iteration 4000000, r = 1.0, Average Regret: 0.431117, Total Time for Last 1000000 Iterations: 306.2602s
Iteration 5000000, r = 1.0, Average Regret: 0.422062, Total Time for Last 1000000 Iterations: 306.9019s
Iteration 6000000, r = 1.0, Average Regret: 0.413919, Total Time for Last 1000000 Iterations: 307.6243s
Experiments completed.
```

In [ ]:
for r_value, df in results.items():
    file_path = os.path.join("results/counter_example", f"regrets_r_{r_value}.csv")
    df.to_csv(file_path, columns=["Iteration", "Average Regret", "x Values"], index=False)
    print(f"Saved regrets for r = {r_value} to {file_path}")

## Plots

In [ ]:
plt.figure(figsize=(10, 6), dpi=150)
for r_value, df in results.items():
    sns.lineplot(x=df["Iteration"], y=df["Average Regret"], label=f"r = {r_value}")

plt.xlabel("Iteration")
plt.ylabel("R_t / t (Average Regret)")
plt.title("Average Regret vs Iteration")
plt.legend()
plt.savefig(os.path.join("results/counter_example/", "average_regret_plot.png"), dpi=150)
plt.close()

In [ ]:
sample_interval = int(1e4)

plt.figure(figsize=(10, 6), dpi=150)
for r_value, df in results.items():
    sampled_df = df[df["Iteration"] % sample_interval == 0]
    sns.lineplot(x=sampled_df["Iteration"], y=sampled_df["Average Regret"], label=f"r = {r_value}")

plt.xlabel("Iteration")
plt.ylabel("R_t / t (Average Regret)")
plt.title("Average Regret vs Iteration (Sampled)")
plt.legend()
plt.grid()
plt.savefig(os.path.join("results/counter_example/", "average_regret_plot_sampled.png"), dpi=150)
plt.close()

In [ ]:
plt.figure(figsize=(10, 6), dpi=150)
for r_value, df in results.items():
    sns.lineplot(x=df["Iteration"], y=df["x Values"], label=f"r = {r_value}")

plt.xlabel("Iteration")
plt.ylabel("x Values")
plt.title("x Values vs Iteration")
plt.legend()
plt.savefig(os.path.join("results/counter_example/", "x_values_plot.png"), dpi=150)
plt.close()

In [ ]:
sample_interval = int(1e4)

plt.figure(figsize=(10, 6), dpi=150)
for r_value, df in results.items():
    sampled_df = df[df["Iteration"] % sample_interval == 0]
    sns.lineplot(x=sampled_df["Iteration"], y=sampled_df["x Values"], label=f"r = {r_value}")

plt.xlabel("Iteration")
plt.ylabel("x Values")
plt.title("x Values vs Iteration (Sampled")
plt.legend()
plt.grid()
plt.savefig(os.path.join("results/counter_example/", "x_values_plot_sampled.png"), dpi=150)
plt.close()